In [1]:
x_auth_token = "9620e9ad2345ba4c0f51a7ecc33ae468"
url = "https://kox947ka1a.execute-api.ap-northeast-2.amazonaws.com/prod/users"

In [2]:
import numpy as np

import requests
import json

In [3]:
header = {"X-Auth-Token" : x_auth_token,
         "Content-Type" : "json",
         }
params = {"problem" : "1" }

res = requests.post(url+'/start', headers = header, params = params)
prob1_key = res.json()['auth_key']

In [5]:
def getLocatedBikes(url, prob1_key):
    headers = {"Authorization" : prob1_key,
               "Content-Type" : "json"}
    bike_cnts = requests.get(url + '/locations', headers = headers)
    return [cnts['located_bikes_count'] for cnts in bike_cnts.json()['locations']]

In [6]:
def getTruckStat(url, prob1_key):
    headers = {"Authorization" : prob1_key,
               "Content-Type" : "json"}
    res = requests.get(url + '/trucks', headers = headers)
    trucks = res.json()['trucks']
    ret = dict()
    for tru in trucks :
        ret.update({tru['id'] : {'loc' : tru['location_id'], 'cnt' : tru['loaded_bikes_count']}})
    return ret

In [7]:
def getZone(idx):
    ret = [idx]
    ret += [idx-1, idx+1, idx-5, idx+5]
    ret += [idx-4, idx+4, idx-6, idx+6]
    return ret

def getZoneBikes(LocBikes, idx):
    zone = getZone(idx)
    ret = {i : LocBikes[i] for i in zone}
    return ret

In [8]:
def getZoneAvg(LocBikes, zone):
    zoneBikes = np.array(LocBikes)
    return np.mean(zoneBikes[zone])

In [9]:
def getMinMaxZone(zoneBikes):
    min_ = 999
    min_idx = 0
    max_ = -1
    max_idx = 0
    for idx in zoneBikes.keys():
        if zoneBikes[idx] < min_ :
            min_ = zoneBikes[idx]
            min_idx = idx
        if zoneBikes[idx] > max_ :
            max_ = zoneBikes[idx]
            max_idx = idx
    return (max_idx, min_idx)

In [10]:
# 0: 6초간 아무것도 하지 않음
# 1: 위로 한 칸 이동
# 2: 오른쪽으로 한 칸 이동
# 3: 아래로 한 칸 이동
# 4: 왼쪽으로 한 칸 이동
# 5: 자전거 상차
# 6: 자전거 하차
def makeRoute(start, end):
    diff = start - end 
    if diff == 1 :
        return [3]
    if diff == -1 :
        return [1]
    if diff == 5 :
        return [4]
    if diff == -5 :
        return [3]
    if diff == 6 :
        return [3, 4]
    if diff == -6 :
        return [2, 1]
    if diff == 4 :
        return [1, 4]
    if diff == -4 :
        return [2, 3]
    if diff == 2 :
        return [3, 3]
    if diff == -2 :
        return [1, 1]
    if diff == 12 : 
        return [4, 4, 3, 3]
    if diff == -12 :
        return [2, 2, 1, 1]
    if diff == 8 :
        return [4, 4, 1, 1]
    if diff == -8 :
        return [2, 2, 3, 3]
    if diff == 0 :
        return []
    if diff == 3 :
        return [4, 1,1]
    if diff == -3 :
        return [2, 3, 3]
    if diff == 7:
        return [4, 3, 3]
    if diff == -7 :
        return [1,1,2]
    if diff == 11 :
        return [4, 4, 3]
    if diff == -11 :
        return [2, 2, 1]
    if diff == 9 :
        return [4, 4, 1]
    if diff == -9 :
        return [2, 2, 3]

In [11]:
def putSimulate(url, prob1_key, commands):
    headers = {"Authorization" : prob1_key,
           "Content-Type" : "json"}
    params = {"commands": commands}
    res = requests.put(url + '/simulate', headers = headers, data = json.dumps(params))
    return res.json()

In [12]:
def getScore(url, prob1_key):
    headers = {"Authorization" : prob1_key,
               "Content-Type" : "json"}
    res = requests.get(url + '/score', headers = headers)
    return res.json()['score']

In [23]:
header = {"X-Auth-Token" : x_auth_token,
         "Content-Type" : "json",
         }
params = {"problem" : "1" }

res = requests.post(url+'/start', headers = header, params = params)
prob1_key = res.json()['auth_key']

In [25]:
# Do Nothing
for i in range(720) : 
    putSimulate(url, prob1_key, commands= [])
getScore(url, prob1_key)

229.94747899159663

In [14]:
# Move Trucks to Center of Zone
# 0: 6초간 아무것도 하지 않음
# 1: 위로 한 칸 이동
# 2: 오른쪽으로 한 칸 이동
# 3: 아래로 한 칸 이동
# 4: 왼쪽으로 한 칸 이동
# 5: 자전거 상차
# 6: 자전거 하차
commands = [
    { "truck_id": 0, "command": [1, 2] },
    { "truck_id": 1, "command": [1,1,1,2,2,2] },
    { "truck_id": 2, "command": [1,2,2,2] },
    { "truck_id": 3, "command": [1,1,1,2] },
    { "truck_id": 4, "command": [1,1,2,2] },
]
putSimulate(url, prob1_key, commands)

{'status': 'ready',
 'time': 1,
 'failed_requests_count': '0.0',
 'distance': '2.0'}

In [15]:
getTruckStat(url, prob1_key)

{0: {'loc': 6, 'cnt': 0},
 1: {'loc': 18, 'cnt': 0},
 2: {'loc': 16, 'cnt': 0},
 3: {'loc': 8, 'cnt': 0},
 4: {'loc': 12, 'cnt': 0}}

In [19]:
zone_idxs = [6, 18, 16, 8, 12]

In [20]:
from tqdm import tqdm

In [21]:
for i in tqdm(range(60 * 12 - 1)):
    bikes = getLocatedBikes(url, prob1_key)
    truck_stat = getTruckStat(url, prob1_key)
    commands = []
    for tru_id, zone_idx in enumerate(zone_idxs) : 
        zone = getZone(zone_idx)
#         zone_avg = getZoneAvg(bikes, zone)
        zone_bikes = getZoneBikes(bikes, zone_idx)
        start, end = getMinMaxZone(zone_bikes)
        now = truck_stat[tru_id]['loc']
        try :
            cmd = makeRoute(now, start) + [5] + makeRoute(start, end) + [6]
            commands.append({"truck_id" : tru_id, "command" : cmd})
        except :
            pass
    res = putSimulate(url, prob1_key, commands)
    if i % 100 == 0 : 
        print(res)

  0%|          | 1/719 [00:00<03:59,  2.99it/s]

{'status': 'ready', 'time': 2, 'failed_requests_count': '0.0', 'distance': '2.3'}


 14%|█▍        | 101/719 [00:27<02:43,  3.77it/s]

{'status': 'ready', 'time': 102, 'failed_requests_count': '7.0', 'distance': '118.0'}


 28%|██▊       | 201/719 [00:56<02:24,  3.59it/s]

...

 84%|████████▎ | 601/719 [03:13<01:20,  1.47it/s]

{'status': 'ready', 'time': 602, 'failed_requests_count': '106.0', 'distance': '630.1'}


 97%|█████████▋| 701/719 [03:44<00:06,  2.77it/s]

{'status': 'ready', 'time': 702, 'failed_requests_count': '116.0', 'distance': '753.9'}


100%|██████████| 719/719 [03:51<00:00,  3.11it/s]


In [137]:
print(tru_id)
print(now, start)

print(makeRoute(now, start)
)
print(start, end)

print(makeRoute(start, end))

1
6 23
None
23 17
[3, 4]


In [101]:
makeRoute(start,end)

In [92]:
makeRoute(now, start)

In [145]:
getLocatedBikes(url, prob1_key)

[2, 3, 2, 1, 2, 3, 2, 2, 2, 4, 0, 2, 2, 1, 2, 2, 2, 1, 2, 0, 2, 2, 1, 2, 2]

In [22]:
getScore(url, prob1_key)

273.611656162465